# Import Libraries

In [34]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## Initialization

In [42]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("scs")
namespaces = ["Kickoff Meeting", "Project Meeting"]

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)
LLM = ChatOpenAI(temperature=0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

# Query

In [38]:
query = "Who is the team lead of the meeting?"

# Generate Embeddings

In [40]:
def get_embeddings(text):
    """
    This function returns a list of the embeddings for a given query
    """
    text_embeddings = EMBEDDINGS.embed_query(text)
    print("Generating Embeddings: Done!")
    return text_embeddings

query_embeddings = get_embeddings(text=query)
print(query_embeddings)

Generating Embeddings: Done!
[-0.016931509599089622, 0.01678033545613289, 0.06843151897192001, -0.029126228764653206, -0.07110226154327393, 0.009435789659619331, 0.008812195621430874, 0.017725175246596336, 0.0025589382275938988, -0.06878425925970078, 0.043966494500637054, -0.012849806807935238, -0.04653645679354668, -0.011930163949728012, -0.023734349757432938, 0.00016770887305028737, -0.031242666766047478, -0.009876714088022709, -0.016742542386054993, 0.013983613811433315, -0.001495207310654223, 0.039154116064310074, -0.041824862360954285, 0.06868347525596619, -0.04648606479167938, 0.0033447290770709515, -0.05316292494535446, 0.02852153219282627, 0.019060546532273293, -0.03560152277350426, -0.005983978509902954, -0.027765661478042603, 0.03398899734020233, -0.022197412326931953, -0.029680535197257996, -0.004884816240519285, 0.004774584900587797, -0.01611264981329441, -0.04860250651836395, -0.004758837632834911, -0.0007432731217704713, -0.05205431580543518, -0.023293424397706985, 0.0080

# Get Namespace

### Semantic Similarity

In [43]:
def get_most_similar_namespace(query, namespaces, threshold=0.8):
    """
    Rank namespaces by semantic similarity to the query.
    """
    query_embeddings = get_embeddings(query)
    print(query_embeddings)

    namespace_embeddings = {ns: get_embeddings(ns) for ns in namespaces}
    print(namespace_embeddings)

    # Compute similarities
    similarities = {
        ns: cosine_similarity([query_embeddings], [embedding])[0][0] for ns, embedding in namespace_embeddings.items()
    }
    print(similarities.items)

    # Rank namespaes by similarity score
    ranked_namespaces = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    print(ranked_namespaces)

    # Check if the top two are close in similarity
    top_two = ranked_namespaces[:2]
    print(top_two)
    if len(top_two) > 1 and abs(top_two[0][1] - top_two[1][1]) < 0.05:
        return None, top_two # Ambiguous case, return for user clarification
    
    return ranked_namespaces[0][0], ranked_namespaces

namespace, ranked = get_most_similar_namespace(query, namespaces)

Generating Embeddings: Done!
[-0.016931509599089622, 0.01678033545613289, 0.06843151897192001, -0.029126228764653206, -0.07110226154327393, 0.009435789659619331, 0.008812195621430874, 0.017725175246596336, 0.0025589382275938988, -0.06878425925970078, 0.043966494500637054, -0.012849806807935238, -0.04653645679354668, -0.011930163949728012, -0.023734349757432938, 0.00016770887305028737, -0.031242666766047478, -0.009876714088022709, -0.016742542386054993, 0.013983613811433315, -0.001495207310654223, 0.039154116064310074, -0.041824862360954285, 0.06868347525596619, -0.04648606479167938, 0.0033447290770709515, -0.05316292494535446, 0.02852153219282627, 0.019060546532273293, -0.03560152277350426, -0.005983978509902954, -0.027765661478042603, 0.03398899734020233, -0.022197412326931953, -0.029680535197257996, -0.004884816240519285, 0.004774584900587797, -0.01611264981329441, -0.04860250651836395, -0.004758837632834911, -0.0007432731217704713, -0.05205431580543518, -0.023293424397706985, 0.0080

### User Clarification Loop

In [ ]:
def clarify_with_user(ambiguous_namespaces):
    """
    Ask the user to clarify when multiple namespaces are similar.
    """
    options = [ns[0] for ns in ambiguous_namespaces]
    print(options)
    print("Did you mean:\n1. {options[0]}\n2. {option[1]}?")

    # Simulate user input for demonstration
    user_choice = int(input("Please choose 1 or 2: "))-1
    return options[user_choice]

### Integration

In [45]:
def resolve_namespace(query, namespaces):
    """
    Resolves the namespace by either selecting the most similar one or prompting the user for clarification.
    """
    namespace, ranked = get_most_similar_namespace(query, namespaces)
    print(namespaces, ranked)

    if namespace:
        print(f"Selected namespace: {namespace}")
        return namespace
    else:
        print("Ambiguity detected!")
        return clarify_with_user(ranked)
    
resolved_namespace = resolve_namespace(query, namespaces)
print(f"Namespace: {resolved_namespace}")

Generating Embeddings: Done!
[-0.016931509599089622, 0.01678033545613289, 0.06843151897192001, -0.029126228764653206, -0.07110226154327393, 0.009435789659619331, 0.008812195621430874, 0.017725175246596336, 0.0025589382275938988, -0.06878425925970078, 0.043966494500637054, -0.012849806807935238, -0.04653645679354668, -0.011930163949728012, -0.023734349757432938, 0.00016770887305028737, -0.031242666766047478, -0.009876714088022709, -0.016742542386054993, 0.013983613811433315, -0.001495207310654223, 0.039154116064310074, -0.041824862360954285, 0.06868347525596619, -0.04648606479167938, 0.0033447290770709515, -0.05316292494535446, 0.02852153219282627, 0.019060546532273293, -0.03560152277350426, -0.005983978509902954, -0.027765661478042603, 0.03398899734020233, -0.022197412326931953, -0.029680535197257996, -0.004884816240519285, 0.004774584900587797, -0.01611264981329441, -0.04860250651836395, -0.004758837632834911, -0.0007432731217704713, -0.05205431580543518, -0.023293424397706985, 0.0080

# Metadata Filtering

## Fuzzy Match

In [24]:
def fuzzy_match(title1, title2, threshold=80):
    """
    Perform a fuzzy match between two titles using RapidFuzz.
    Returns True if the similarity score is above the threshold.
    """
    similarity_score = fuzz.partial_ratio(title1.lower(), title2.lower())
    return similarity_score >= threshold

## Extract Metadata

In [27]:
def extract_metadata_from_query(query):
  prompt = f"""
  You are a helpful assistant. Extract the meeting title and the meeting date from the following query.
  If the meeting title or date is not explicitly mentioned, return 'unknown'.
  If the date is mentioned as word, it should be formatted as 'YYYY-MM-DD'

  Query: {query}

  Provide the meeting title and date as a Python dictionary in this format:
  {{"meeting_title": "title_here", "date": "date_here"}}
  """

  response = LLM.invoke(prompt)
  metadata_str = response.content.strip()
  metadata_dict = ast.literal_eval(metadata_str)
  return metadata_dict

metadata = extract_metadata_from_query(query)
print(metadata)

{'meeting_title': 'project meeting', 'date': 'unknown'}


# Query Pinecone Index

In [46]:
def query_pinecone_index(query_embeddings, meeting_title, date, top_k=3, include_metadata=True):
    """
    Query a Pinecone index.
    """
    filter_conditions = {}
    if date.lower() != 'unknown':
      filter_conditions['date'] = date
    # if meeting_title.lower() != 'unknown':
    #   filter_conditions['title'] = meeting_title
    # if date.lower() != 'unknown':
    #   filter_conditions['date'] = date

    query_response = index.query(
        vector=query_embeddings,
        filter=filter_conditions,
        top_k=top_k,
        include_metadata=include_metadata,
        namespace="Project Meeting") # Filter based on metadata

    filtered_matches = []
    for match in query_response['matches']:
      if 'metadata' in match and 'title' in match['metadata']:
        metadata_title = match['metadata']['title']
        if fuzzy_match(meeting_title, metadata_title):
          filtered_matches.append(match)

    if not filtered_matches:
      return query_response

    query_response['matches'] = filtered_matches

    print("Querying Pinecone Index: Done!")
    return query_response

answers = query_pinecone_index(query_embeddings=query_embeddings, meeting_title=resolved_namespace, date="unknown")
print(answers)

Querying Pinecone Index: Done!
{'matches': [{'id': 'e773312289360121c6069407784bbc732f6f8d4505330eb28deaaa0add6a0cd7',
              'metadata': {'date': '2024-12-01',
                           'text': 'Czech: Hello my name is Czech.\n'
                                   'Gian: Hello my name is Gian.\n'
                                   'Shaundyl: Hello my name is Shaundyl.\n'
                                   'Czech (Team Lead): Alright, everyone, '
                                   'thanks for joining today’s meeting. We '
                                   'have about 10 minutes to go over the final '
                                   "details before the product launch. Let's "
                                   'start with the progress update. Bob, how '
                                   'are we doing on the development front?',
                           'title': 'Project Meeting'},
              'score': 0.477952331,
              'values': []},
             {'id': 'ae52e04

# Combining Text from Multiple Document Matches

In [31]:
text_answer = " ".join([doc['metadata']['text'] for doc in answers['matches']])
print(text_answer)

Czech: Hello my name is Czech.
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?
Shaundyl (Developer): Things are looking good. We’ve implemented all the major features. I’m currently working on the final round of bug fixes. I should be done with it by tomorrow, but I need the QA team to give it another pass afterward. Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.
Czech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?
Shaundyl: Nothing from my side. I’ll update you if any blockers come up.

## Prompt

In [32]:
prompt = f"""You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.
        Context: {text_answer}
        Question: {query}"""

print(prompt)

You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.
        Context: Czech: Hello my name is Czech.
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?
Shaundyl (Developer): Things are looking good. We’ve implemented all the major features. I’m currently working on the final round of bug fixes. I should be done with it by tomorrow, but I need the QA team to give it another pass afterward. Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the co

# LLM

In [33]:
def better_query_response(prompt):
    """
    This function returns a better response using LLM
    """
    better_answer = LLM.invoke(prompt)
    print("Generating Better Response: Done!")
    return better_answer

final_answer = better_query_response(prompt=prompt)
print(final_answer.content)

Generating Better Response: Done!
Czech is the team lead of the project meeting.


# Chatbot Response

In [12]:
def Chatbot(query, meeting_title=None, date=None):
    print(query)
    metadata = extract_metadata_from_query(query)
    print(metadata)
    meeting_title = metadata.get('meeting_title', 'unknown')
    date = metadata.get('date', 'unknown')

    query_embeddings = get_query_embeddings(query=query)

    answers = query_pinecone_index(
        query_embeddings=query_embeddings,
        meeting_title=meeting_title,
        date=date
        )
    print(answers)

    text_answers = " ".join([doc['metadata']['text'] for doc in answers['matches']])
    print(text_answers)
    final_answer = better_query_response(prompt=prompt)
    return final_answer.content

response = Chatbot(query)
print(response)

What are the keypoints from the project meeting?


SyntaxError: invalid syntax (<unknown>, line 1)